<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/Tarea2IO/Metodo-Grafico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Funcion

In [ ]:
class Condicion:
  MENOR_IGUAL=0
  MAYOR_IGUAL=1
  MENOR=2
  MAYOR=3
  def __init__(self,coeficiente:float,corte:float, tipo:int=MENOR, en='y'):
    '''
    Crea una condicion de la forma y (<=)(>=)(<)(>) coeficiente*x+corte
    En caso de que el atributo "en='x'" se va a ignorar el coeficiente y
    determinará una expresion de la forma x (<=)(>=)(<)(>) corte.

    '''
    self.coeficiente=coeficiente
    self.corte=corte
    self.tipo=tipo
    self.en=en
    self.funcion=None

  def __add__(self,other):
    '''
    Estas operaciones no se supone deban usarse por el usuario, son solo para poder hacer calculos sobre las intersecciones entre dos funciones.
    '''
    return Condicion(self.coeficiente+other.coeficiente,self.corte+other.corte,self.tipo)

  def __sub__(self,other):
    '''
    Estas operaciones no se supone deban usarse por el usuario, son solo para poder hacer calculos sobre las intersecciones entre dos funciones.
    '''
    return Condicion(self.coeficiente-other.coeficiente,self.corte-other.corte,self.tipo)

  def __repr__(self):
    #Muestra la funcion
    if self.en=='y':
      return "y {} {}x+({})".format(["<=",">=","<",">"][self.tipo],self.coeficiente,self.corte)
    return "x {} ({})".format(["<=",">=","<",">"][self.tipo],self.corte)

  def condicion(valor:float, tipo:int, en:str):
    '''
    Es un método que permite hacer una condicion simple por ejemplo y<3 o x>=5
    '''
    return Condicion(0,valor,tipo,en)

  def getCorte(self):
    #Retorna los puntos de corte con el eje horizontal y el eje vertical
    return ((0,self.corte),((-self.corte)/self.coeficiente,0))

  def evaluar(self,x):
    #Evalua un valor como si fuera una igualdad
    if self.en=='x':
      salida=[]
      for i in x:
        if i==self.corte:
          salida.append([0,1])
        else:
          salida.append(np.nan())
        
    if self.funcion==None:
      self.funcion=lambda x:self.coeficiente*x+self.corte
    return self.funcion(x)

  def evaluarCondicion(self,x,y=0):
    #evalua si se cumple la condicion para un numero dado, redondeado a 5 decimales.
    f=[lambda a,b: a<=b,lambda a,b: a>=b][self.tipo%2]
    if self.en=='y':
      return f(round(y,5),round(self.evaluar(x),5))
    else:
      return f(x,self.corte)

  def getInterseccion(self, other):
    #Calcula la interseccion de dos rectas
    if ((self.en=='x' or other.en=='x') and not(self.en=='x' and other.en=='x')):
      a,b=(self,other) if self.en=='x' else (other,self)
      return a.corte, b.evaluar(a.corte)
    if (self.en=='y' and other.en=='y'):
      p=(self-other).getCorte()[1][0]
      return p,self.evaluar(p)
    return None


#Solver

In [ ]:
class Solver:
  def __init__(self, condiciones):
    '''
    Solver se encargará de hallar los vertices y de optimizar la funcion objetivo,
    sin embargo no se va a pedir la funcion objetivo en el constructor
    '''
    self.condiciones=condiciones
  def __puntos(self):
    #retorna todas las intersecciones, pues se quiere poder utilizarlas para optimizar la funcion objetivo
    k=self.condiciones
    p=[]
    for i in range(len(k)):
      for j in range(i+1,len(k)):
        try:
          p.append(k[i].getInterseccion(k[j]))
        except:
          pass
    return p

  def vertices(self):
    p=self.__puntos()
    s=p[:]
    for i in self.condiciones:
      for j in p:
        if (not i.evaluarCondicion(j[0],j[1])):
          if j in s:
            s.remove(j)
    return s
  
  def optimizar(self,funcionObjetivo=lambda x,y: x+y, modo='max'):
    puntos=self.vertices()
    valores=[]
    for i in puntos:
      valores.append(funcionObjetivo(i[0],i[1]))
    valores=np.array(valores)
    if modo=='max':
      return puntos[np.argmax(valores)], valores.max()
    else:
      return puntos[np.argmax(-valores)], valores.min()
    


##Prueba

In [ ]:
condiciones=[Condicion(-2,3,Condicion.MAYOR),
             Condicion(1,0,Condicion.MENOR_IGUAL),
             Condicion(0,1/2,Condicion.MAYOR),
             Condicion.condicion(2,Condicion.MENOR,'x')]

for i in condiciones:
  print(i)
w=Solver(condiciones)
print(w.vertices())
x=np.linspace(0,2,10)
plt.figure(figsize=(7,7))
for i in condiciones:
  try:
    plt.plot(x,i.evaluar(x))
  except:
    pass

y=np.array(w.vertices()).T
if len(y)>0:
  plt.scatter(y[0],y[1],color="red")
plt.grid()
plt.show()

In [ ]:
w.optimizar(lambda x,y:x**2+np.log(y), modo='min')

#Graficador

In [ ]:
xlims=(0,3)
ylims=(0,3)

X__=[np.linspace(xlims[0],xlims[1],300),np.linspace(ylims[0],ylims[1],300)]
Y__=None
for i in condiciones:
  temp=np.array([[i.evaluarCondicion(x0,x1) for x0 in X__[0]] for x1 in X__[1]],dtype="int64")
  if type(Y__)==type(None):
    Y__=-temp
    continue
  Y__-=temp

plt.figure(figsize=(7,7))
plt.pcolormesh(X__[0],X__[1],Y__,cmap="binary")
plt.xlim(xlims[0],xlims[1])
plt.ylim(ylims[0],ylims[1])
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
cmap = ListedColormap(["white","cyan"])

In [ ]:
plt.figure(figsize=(7,7))
plt.pcolormesh(X__[0],X__[1],-np.minimum(Y__.min()+1,Y__),cmap=cmap)
plt.xlim(xlims[0],xlims[1])
plt.ylim(ylims[0],ylims[1])

In [ ]:
plt.figure(figsize=(7,7))
plt.pcolormesh(X__[0],X__[1],-np.minimum(Y__.min()+1,Y__),cmap=cmap)
plt.xlim(xlims[0],xlims[1])
plt.ylim(ylims[0],ylims[1])

x=X__[0]
for i in condiciones:
  try:
    plt.plot(x,i.evaluar(x))
  except:
    pass

y=np.array(w.vertices()).T
if len(y)>0:
  plt.scatter(y[0],y[1],color="red")
plt.grid()